In [ ]:
#Import necessary library to analyst the data 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#import email 
email = pd.read_csv('../data/email.csv')

In [ ]:
email.head()

In [ ]:
email.columns

In [ ]:
#Convert the data format 
email['date'] = pd.to_datetime(email['date'], format='%m/%d/%Y %H:%M:%S')

In [ ]:
# Frequency of emails by user
user_frequency = email['user'].value_counts()

# Number of attachments analysis (if the number of attachments is higher it should notice risk
attachment_analysis = email['attachments'].value_counts()

In [ ]:
user_frequency

In [ ]:
email_ex = email.query("user == 'ABC0174'")

In [ ]:
email_ex

In [ ]:
email_ex = email.query("user == 'NWT0098'")
email_ex

In [ ]:
#@dtaa.com is company email 

def contains_dtaa(row):
    relevant_columns = ['to', 'cc', 'bcc']
    for col in relevant_columns:
        if pd.notnull(row[col]):
            # Split the column value by semicolon and check each email
            emails = row[col].split(';')
            if any('@dtaa.com' in email.strip() for email in emails):
                return True
    return False

# Create the 'not_dtaa' column
email['not_dtaa'] = ~email.apply(contains_dtaa, axis=1)

In [ ]:
#Check values fo email
email['not_dtaa'].value_counts()

In [ ]:
#remove @dtaa.com because 
email_filtered = email[email['not_dtaa'] == True].reset_index(drop=True)

In [ ]:
email_filtered.query("user == 'XHW0498'")

In [ ]:
user_frequency = email_filtered['user'].value_counts()

In [ ]:
user_frequency.mean()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(user_frequency, kde=True, bins=30, color='blue')
plt.title('User Frequency Distribution', fontsize=16)
plt.xlabel('Email Count per User', fontsize=14)
plt.ylabel('Frequency', fontsize=14);

In [ ]:
#find the top users to 
top_users = email_filtered['user'].value_counts().head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_users.index, y=top_users.values)
plt.title('Top 10 Users by Email Activity')
plt.xlabel('User')
plt.ylabel('Email Count')
plt.show()

In [ ]:
# impute Missing values in 'cc' and 'bcc'
email_filtered['cc'] = email_filtered['cc'].fillna('')
email_filtered['bcc'] = email_filtered['bcc'].fillna('')

#summary number of recipient from to cc, bcc
email_filtered['num_recipients'] = email_filtered['to'].str.count(';') + email_filtered['cc'].str.count(';') + email_filtered['bcc'].str.count(';') + 1
email_filtered['hour'] = email_filtered['date'].dt.hour
email_filtered['day_of_week'] = email_filtered['date'].dt.dayofweek 

# Define working hours and working days
working_hours_start = 9 # 9 AM
working_hours_end = 17  # 5 PM
working_days = [0, 1, 2, 3, 4]  # Monday=0, Sunday=6

#is working_time columns 
email_filtered['is_working_time'] = (
    (email_filtered['hour'] >= working_hours_start) & 
    (email_filtered['hour'] < working_hours_end) & 
    email_filtered['day_of_week'].isin(working_days))

# Drop rows sent during working hours and working days
email_filtered = email_filtered[~email_filtered['is_working_time']]

# Drop the column if no longer needed
email_filtered.drop(columns=['is_working_time'], inplace=True)

In [ ]:
email_filtered

In [ ]:
# Calculate the mean frequency
mean_frequency = user_frequency.mean()

# Filter users who send more emails than the mean
users_above_mean = user_frequency[user_frequency > mean_frequency]

# Filter include only these users
filtered_email = email_filtered[email_filtered['user'].isin(users_above_mean.index)]

In [ ]:
filtered_email

In [ ]:
# Predefined list of public domains
public_domains = ['gmail.com', 'yahoo.com', 'outlook.com', 'hotmail.com', 'msn.com', 'icloud.com', 'aol.com', 'live.com', 'protonmail.com', 'zoho.com', 'mail.com', 'gmx.com']

def contains_public_domain(row):
    relevant_columns = ['to', 'cc', 'bcc']
    for col in relevant_columns:
        if pd.notnull(row[col]):
            emails = row[col].split(';')
            for email in emails:
                domain = email.strip().split('@')[-1]
                if domain in public_domains:
                    return True
    return False

# create the 'is_public_domain' column
filtered_email['is_public_domain'] = filtered_email.apply(contains_public_domain, axis=1)


In [ ]:
filtered_email['is_public_domain'].value_counts()

In [ ]:
# Separate the DataFrame into two based on the 'is_public_domain' column
public_domain_df = filtered_email[filtered_email['is_public_domain'] == True]
non_public_domain_df = filtered_email[filtered_email['is_public_domain'] == False]

In [ ]:
# Extract domains from email addresses
def extract_domains(emails):
    if pd.notnull(emails):  
            return [email.strip().split('@')[-1] for email in emails.split(';') if '@' in email]
    return []

# 'non_public_domain_df' DataFrame
non_public_domain_df['to_domains'] = non_public_domain_df['to'].apply(extract_domains)
non_public_domain_df['cc_domains'] = non_public_domain_df['cc'].apply(extract_domains)
non_public_domain_df['bcc_domains'] = non_public_domain_df['bcc'].apply(extract_domains)

# Combine all domains into one list
all_non_public_domains = non_public_domain_df['to_domains'].explode().tolist() + non_public_domain_df['cc_domains'].explode().tolist() + non_public_domain_df['bcc_domains'].explode().tolist()

In [ ]:
all_non_public_domains

In [ ]:
# List of domains with vendors
vendor_domains = ['boeing.com', 'raytheon.com', 'hp.com', 'harris.com', 'northropgrumman.com', 'lockheedmartin.com']

def is_vendor_email(email):
    if pd.notnull(email):
        domain = email.split('@')[-1]
        return domain in vendor_domains
    return False

for index, row in filtered_email.iterrows():
    for col in ['to', 'cc', 'bcc']:
        emails = row[col].split(';') if pd.notnull(row[col]) else []
        if any(is_vendor_email(email) for email in emails):
            filtered_email.at[index, 'is_vendor'] = True
            break  

filtered_email_no_vendors = filtered_email[filtered_email['is_vendor'] == False]

In [ ]:
#Data is clean
filtered_email_no_vendors

In [ ]:
filtered_email_no_vendors['user'].value_counts()

In [ ]:
# Further analysis on filtered_email_no_vendors and public_domain_df to identify the true positive